# HW-7 SVM
Wyatt Blair

SID: 10420296

2/26/24


## 9.1

The “breast cancer dataset” (wisc_bc_ContinuousVar.csv) in CANVAS was obtained from the University of Wisconsin Hospitals, Madison from Dr. William H. Wolberg. Use the SVM methodology , to develop a classification model for the Diagnosis.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

tqdm.pandas()

In [12]:
dataset = pd.read_csv('../data/wisc_bc_ContinuousVar.csv', index_col='id')
dataset

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820


In [13]:
def min_max_normalization(series):

    minimum, maximum = series.min(), series.max()
    normalized_data = (series - minimum) / (maximum - minimum)

    return normalized_data

def z_score_standardization(series):

    mean, std = series.mean(), series.std()
    normalized_data = (series - mean) / std

    return normalized_data

In [14]:
target = 'diagnosis'
features = dataset.columns.to_list(); features.remove(target)

In [15]:
normalized_dataset = dataset.copy()
normalized_dataset[features] = min_max_normalization(normalized_dataset[features])

train = normalized_dataset.sample(frac=0.8)
test = normalized_dataset.drop(index=train.index)

___
### Test out sklearn's implementation

In [16]:
from sklearn.svm import SVC

In [17]:
svc = SVC()

X, y = train[features], train[target]
svc.fit(X, y)

test['prediction_sk'] = svc.predict(test[features])
test['correct_sk'] = test['prediction_sk'] == test[target]
correct_counts = test['correct_sk'].value_counts()

acc = correct_counts[True] / len(test.index)
print('Accuracy: %1.2f%%' % (acc*100))

Accuracy: 98.25%


___
### From scratch

In [18]:
class SVM:

    def __init__(self, X: pd.DataFrame, y: pd.Series, num_epochs, learning_rate: float, expanding_rate: float, retracting_rate: float):

        # load data
        self.X = X
        self.y = y

        # set some constants
        self.num_epochs = int(num_epochs)
        self.lr = learning_rate
        self.er = expanding_rate
        self.rr = retracting_rate

        # make sure constants are expected values
        assert self.rr > 1
        assert self.er < 1

        # infer features and target
        self.features = self.X.columns.to_list()
        self.target = self.y.name

        # build reference dict for class values
        self.class_values = {
            i: val 
            for i, val in zip([1, -1], self.y.unique())
        }
        assert len(self.class_values) == 2

        # inverse of class_values dict
        self.numeric_values = {
            val: i 
            for i, val in self.class_values.items()
        }

        # constants pertaining to shape of data
        self.N = len(self.X.index)
        self.M = len(self.features)

        # core model variables randomly initialized
        self.w = np.random.random(size=self.M)
        self.b = np.random.random()

    def __call__(self, x):

        return self.predict(x)

    def predict(self, x):
        
        # determine whether x, the feature vector,
        # falls above or below the hyper-plane
        # classify it accordingly

        # calculate model value
        model_val = np.dot(self.w, x.T) + (self.b)

        # if value < 0 then it is the "-1" class
        if model_val < 0:
            return self.class_values[-1]
        
        # if value >= 0 then it is the "1" class
        else:
            return self.class_values[1]

    def expand_road(self, x, y):

        self.w += x * y * self.lr * self.er
        self.b +=     y * self.lr * self.er
    
    def retract_road(self, x, y):

        self.w += x * y * self.lr * self.rr
        self.b +=     y * self.lr * self.rr

    def fit(self):

        for epoch in tqdm(range(self.num_epochs), desc='Training SVM', total=self.num_epochs):
            
            # pick a random datapoint
            x = self.X.sample()
            y = self.y.loc[x.index].values[0]

            # make prediction with current model
            pred = self.predict(x)
            
            # convert y and pred into numeric values
            y_numeric = self.numeric_values[y]
            pred_numeric = self.numeric_values[pred]

            # if prediction was correct
            if pred == y:
                
                # if the numeric predicition was outside the range of
                # (-1, 1) then it means the "street" is too wide and
                # needs to be retracted
                street_too_wide = -1 < pred_numeric < 1

                # retract street
                if street_too_wide: self.retract_road(x, y_numeric)
                else:
                    self.w *= self.er
                    self.b *= self.er
            
            # if prediction was not correct expand the road
            else: self.expand_road(x, y_numeric)





In [23]:
svm = SVM(
    X=train[features],
    y=train[target],
    num_epochs=1e3,
    learning_rate=0.5,
    expanding_rate=0.99,
    retracting_rate=1.01
)

svm.fit()

Training SVM: 100%|██████████| 1000/1000 [00:00<00:00, 1750.47it/s]


In [24]:
test['prediction_scratch'] = test[features].apply(svm, axis=1)
test['correct_scratch'] = test[target] == test['prediction_scratch']
correct_counts = test['correct_scratch'].value_counts()

acc = correct_counts[True] / len(test.index)
print("Accuracy: %1.2f%%" %(acc*100))

Accuracy: 90.35%
